In [ ]:
import importlib
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Explore the amount of nans

In [ ]:
# Explore nans
file_path = './data/Dane/1year.csv'
data = pd.read_csv(file_path, na_values='?')
print('In total we have {} rows and {} features'.format(*data.shape))
na_data = pd.isnull(data)
num_nans = na_data.sum(axis=1)
plt.figure(1)
plt.hist(num_nans, bins=num_nans.max())
plt.title('nans per row')
plt.figure(2)
plt.hist(na_data.sum(axis=0), bins=50)
plt.title('nans per feature')
plt.show()

In [ ]:
# What happens if we through away the two features with highest frequency nans?
na_data_dropped_some = na_data.drop(['attr37', 'attr21'], axis=1)
num_nan_dropped = na_data_dropped_some.sum(axis=1)
share_with_nan = (num_nan_dropped != 0).mean()
print('Then we get {} share with at least one nan.'.format(share_with_nan))